In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [11]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']

In [12]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09] #feature weights
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64(InitialDate, dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [13]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        #print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [14]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y))]# |
    '''
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    '''
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=35, weights='distance', 
                               metric='manhattan')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [15]:
def run_validation():
    df_train = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20 
    
    print('Preparing train data', flush=True)
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    prediction_result = run_prediction(df_train, df_valiation, n_cell_x, n_cell_y, 5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)


In [9]:
print('week', flush = True)
InitialDate = '2014-01-01T01:01'
run_validation()
#0.539376

InitialDate = '2014-01-02T01:01'
run_validation()
#0.539470

InitialDate = '2014-01-03T01:01'
run_validation()
#0.539267

InitialDate = '2014-01-04T01:01'
run_validation()
#0.539261

InitialDate = '2014-01-05T01:01'
run_validation()
#0.539346

InitialDate = '2014-01-06T01:01'
run_validation()
#0.539201

print('hour', flush = True)
InitialDate = '2014-01-01T01:01'
run_validation()
#0.539376

InitialDate = '2014-01-02T01:30'
run_validation()
#0.539424

InitialDate = '2014-01-03T02:00'
run_validation()
#0.539291

InitialDate = '2014-01-04T02:30'
run_validation()
#0.539268

InitialDate = '2014-01-05T03:00'
run_validation()
#0.539269

InitialDate = '2014-01-06T03:30'
run_validation()
#0.539143

InitialDate = '2014-01-06T00:00'
run_validation()
#0.539216

week
Preparing train data
Elapsed time overall: 102.08990478515625 seconds 0
Elapsed time overall: 110.56617999076843 seconds 1
Elapsed time overall: 106.2749252319336 seconds 2
Elapsed time overall: 109.42328143119812 seconds 3
Elapsed time overall: 107.29022097587585 seconds 4
Elapsed time overall: 109.52788877487183 seconds 5
Elapsed time overall: 116.02365469932556 seconds 6
Elapsed time overall: 121.92425656318665 seconds 7
Elapsed time overall: 115.14268326759338 seconds 8
Elapsed time overall: 106.2404248714447 seconds 9
count    4417684.000000
mean           0.539376
std            0.451266
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
Elapsed time overall: 104.04574012756348 seconds 0
Elapsed time overall: 115.03219652175903 seconds 1
Elapsed time overall: 104.71317219734192 seconds 2
Elapsed time overall: 93.08326196670532 seconds 3
Elapsed time overall: 

In [16]:
InitialDate = '2014-01-06T04:00'
run_validation()
#0.539122

InitialDate = '2014-01-06T04:30'
run_validation()
#0.539120

InitialDate = '2014-01-06T05:00'
run_validation()
#0.539096

InitialDate = '2014-01-06T05:30'
run_validation()
#0.539076

InitialDate = '2014-01-06T06:00'
run_validation()
#0.539045

InitialDate = '2014-01-06T06:30'
run_validation()
#0.539026

InitialDate = '2014-01-06T07:00'
run_validation()
#0.539064

InitialDate = '2014-01-06T07:30'
run_validation()
#0.539051

InitialDate = '2014-01-06T08:00'
run_validation()
#0.539061

InitialDate = '2014-01-06T08:30'
run_validation()
#0.539037

InitialDate = '2014-01-06T09:00'
run_validation()
#0.539023

InitialDate = '2014-01-06T09:30'
run_validation()
#0.538975

InitialDate = '2014-01-06T10:00'
run_validation()
#0.539016

InitialDate = '2014-01-06T10:30'
run_validation()
#0.538981

InitialDate = '2014-01-06T11:00'
run_validation()
#0.539011

InitialDate = '2014-01-06T11:30'
run_validation()
#0.539083

Preparing train data
count    4417684.000000
mean           0.539122
std            0.451215
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
count    4417684.000000
mean           0.539120
std            0.451206
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
count    4417684.000000
mean           0.539096
std            0.451212
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
count    4417684.000000
mean           0.539076
std            0.451212
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
count    441768

Validation result: 0.539376

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)